In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

%matplotlib inline

In [ ]:
def readParticleConfig(path,ConfigurationNumber):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = 1+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    NumberOfAParticles = int(FirstRow.split()[5].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[7].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            BIndex += 1
    return APositions, BPositions

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

dataPath = getDirectoryPathFromDialog('')

def readParticleConfigFromDialog(ConfigurationNumber):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber)

In [ ]:
def readStructureFactorValues(path, number_of_skipped_rows):
    return np.loadtxt(path, dtype=float, skiprows = number_of_skipped_rows)

def readStructureFactorValuesFromDialog(skiprows):
    return readStructureFactorValues(getFilePathFromDialog(dataPath),skiprows)

## Plot pot energy series

In [ ]:
def plotMultiplePotEnergySeries():
    Directory = getDirectoryPathFromDialog(dataPath);
    AllPotEnergySeries = sorted(glob.glob(Directory+"/PotEnergy*"))
    NumberOfSeries = len(AllPotEnergySeries)
    Temperature = re.search(r'(?<=T=)[\d]*.[\d]*',Directory).group()
    Fig,Ax = plt.subplots(NumberOfSeries,2,squeeze=False, figsize=(18,5*NumberOfSeries))
    Fig.suptitle("T="+Temperature,y=0.95)
    Fig.tight_layout(rect=[0, 0.03, 1, 0.95])    
    for i in range(NumberOfSeries):
        RunNumber = re.search(r'(?<=_)[\d]*(?=(.dat))',AllPotEnergySeries[i]).group()
        PotSeries = np.loadtxt(glob.glob(Directory+"/PotEnergy*_"+RunNumber+".dat")[0], dtype=float, skiprows = 1)
        PotEnergyAverage = np.mean(PotSeries)
        Ax[i,0].plot(PotSeries,'.')
        Ax[i,1].hist(PotSeries,bins = 20, density = True)
        Ax[i,1].vlines(PotEnergyAverage,0,0.004,color='g',label="PotEnergyAvg={}".format(round(PotEnergyAverage,2)))
        Ax[i,0].set_title("Run "+RunNumber)
        Ax[i,1].legend()

In [ ]:
plotMultiplePotEnergySeries()

## Structure factor plotting

In [ ]:
StructureFactorResults = readStructureFactorValuesFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,1],'.',label=r'$S_{AA}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,2],'+',label=r'$S_{BB}$')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('strcture factor value')
Ax[0,0].legend()
Ax[0,0].set_title(r'$S_{AA}$'+' and '+r'$S_{BB}$' + ' as function of k')
#Ax[0,0].set_xlim(0,1)
#Ax[0,0].set_ylim(0.71,0.74)
#Fig.savefig('SAA_SBB'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,3],'.')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('$S_{AB}(k)$')
Ax[0,0].set_title(r'$S_{AB}$')
#Ax[0,0].set_xlim(0,1)
#Fig.savefig('SAB'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,4],'.')
Ax[0,0].set_xlabel(r'$k[\sigma^{-1}]$')
Ax[0,0].set_ylabel('$S_{cc}(k)$')
Ax[0,0].set_title(r'$S_{cc}$')
Ax[0,0].set_xlim(0,1)
#Fig.savefig('Scc'+'.png',format='png')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi = 200)
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,1],'-',label=r'$S_{AA}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,2],'-',label=r'$S_{BB}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,3],'-',label=r'$S_{AB}$')
Ax[0,0].plot(StructureFactorResults[:,0], StructureFactorResults[:,4],'-',label=r'$S_{cc}$')
Ax[0,0].legend()

## Plot multiple Sccs

In [ ]:
def plotMultipleSccs(Tc,kSquareMin,kSquareMax):
    Directory = getDirectoryPathFromDialog(dataPath)
    files = glob.glob(Directory+"/StructureFactors_*")
    Fig, Ax = plt.subplots(2,1,squeeze=False, figsize=(7,15),dpi=100)
    Fig.suptitle(r'multiple Scc')
    for file in sorted(files):
        Temperature = float(re.search(r'(?<=T=)[\d]*.[\d]*',file).group())
        StructureFactorResults = readStructureFactorValues(file,1)
        SccInverse = 1/(StructureFactorResults[:,4])
        kSquared = (StructureFactorResults[:,0])**2
        Mask = ((kSquared > kSquareMin) & (kSquared < kSquareMax))
        i = 0 if (Temperature < Tc) else 1
        Ax[i,0].plot(kSquared[Mask], SccInverse[Mask],'.',label='T='+str(Temperature))
    for i in range(2):
        Ax[i,0].legend()
        Ax[i,0].set_xlabel(r'$\left( k\sigma \right)^{2}$')
        Ax[i,0].set_ylabel(r'$\left( S_{cc}(k) \right)^{-1}$')
    Ax[0,0].set_title(r'$T<T_c$')
    Ax[1,0].set_title(r'$T>T_c$')

In [ ]:
plotMultipleSccs(0.7469,0.025,0.2)

## Ornstein-Zernike-fit

In [ ]:
def LinearFunction(x,m,n):
    return m*x+n;

In [ ]:
def PowerLaw(x,Amp,Exponent):
    return Amp*x**(-Exponent)

In [ ]:
def OrnsteinZernike(k,rohchikBT,xi):
    return (rohchikBT*(1.0+(k*xi)**2)**(-1))

In [ ]:
StructureFactorResults = readStructureFactorValuesFromDialog(1)

In [ ]:
LowerIndex = 12
UpperIndex = 60

In [ ]:
popt, pcov = curve_fit(lambda k,xi: OrnsteinZernike(k,29.2688,xi),StructureFactorResults[LowerIndex:UpperIndex,0],StructureFactorResults[LowerIndex:UpperIndex,4],p0=[1],maxfev=50000,bounds=([0],[np.inf]))
print("Parameters found: xi=", popt[0])

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
Ax[0,0].plot(StructureFactorResults[LowerIndex:UpperIndex,0], StructureFactorResults[LowerIndex:UpperIndex,4],'.')
Ax[0,0].plot(StructureFactorResults[LowerIndex:UpperIndex,0],OrnsteinZernike(StructureFactorResults[LowerIndex:UpperIndex,0],29.2688,*popt))
Ax[0,0].set_xlabel(r'$\left( k\sigma \right)$')
Ax[0,0].set_ylabel(r'$\left( S_{cc}(k) \right)$')
Ax[0,0].set_title(r'$S_{cc}$')
#Ax[0,0].set_xlim(0,1.5)

In [ ]:
def computeMultipleCorrelationLengths(Tc,kMin,kMax,rohchikBTDict):
    Directory = getDirectoryPathFromDialog(dataPath)
    files = glob.glob(Directory+"/StructureFactors_*")
    Temperatures = []
    RedTemperatures = []
    CorrelationLengths = []
    Susceptibilities = []
    i = 0
    Fig, Ax = plt.subplots(len(files),1,squeeze=False, figsize=(7,70),dpi=100)
    Fig.subplots_adjust(hspace=0.3)
    for file in sorted(files):
        Density = float(re.search(r'(?<=AvgDens=)[\d]*.[\d]*',file).group())
        Temperature = float(re.search(r'(?<=T=)[\d]*.[\d]*',file).group())
        Temperatures.append(Temperature)
        rohchikBT = rohchikBTDict[Temperature]
        Susceptibilities.append(rohchikBT/(Temperature*Density))
        RedTemperatures.append(abs(Temperature-Tc)/Tc)
        StructureFactorResults = readStructureFactorValues(file,1)
        k = StructureFactorResults[:,0]
        Mask = ((k > kMin) & (k < kMax))
        popt, pcov = curve_fit(lambda k,xi: OrnsteinZernike(k,rohchikBT,xi),k[Mask],StructureFactorResults[:,4][Mask],p0=[1],maxfev=50000,bounds=([0],[np.inf]))
        CorrelationLength = popt[0]
        CorrelationLengths.append(CorrelationLength)
        Ax[i,0].plot(k[Mask], StructureFactorResults[:,4][Mask],'.')
        Ax[i,0].plot(k[Mask], OrnsteinZernike(k[Mask],rohchikBT,*popt),label=r'$roh$'+r' $\chi k_B T$='+str(rohchikBT)+r', $\frac{\xi}{\sigma}=$'+str(CorrelationLength))
        #Ax[i,0].text(0.05, 0.75,'m={:.2f}, n={:.3e}'.format(popt[0],popt[1]), transform=Ax[i,0].transAxes, fontsize=14,verticalalignment='top')
        Ax[i,0].set_xlabel(r'$\left( k\sigma \right)$')
        Ax[i,0].set_ylabel(r'$\left( S_{cc}(k) \right)$')
        Ax[i,0].set_title(r'T='+str(Temperature))
        Ax[i,0].legend(loc='upper left')
        #Ax[i,0].set_xlim(0,1.5)
        #Ax[i,0].set_xscale('log')
        #Ax[i,0].set_yscale('log')
        i+=1
    #Fig.savefig("plots/structure_factor_fits_ornstein_zernike.png",format='png',dpi=200)
    return Temperatures, RedTemperatures, Susceptibilities, CorrelationLengths

In [ ]:
def getrohchikBTDict():
    TvsSus = readStructureFactorValuesFromDialog(1)
    SusDict = {}
    for i in range(len(TvsSus[:,0])):
        SusDict[TvsSus[i,0]] = TvsSus[i,1]
    return SusDict

In [ ]:
rohchikBTDict = getrohchikBTDict()

In [ ]:
Temperatures, RedTemperatures, Susceptibilities, CorrelationLengths = computeMultipleCorrelationLengths(0.7469,0.2,0.6,rohchikBTDict)

In [ ]:
Tcrit = 0.7469
Mask = ((np.array(Temperatures) > 0.75) & (np.array(Temperatures) < 0.85))

In [ ]:
Results = np.column_stack([Temperatures,RedTemperatures,Susceptibilities,CorrelationLengths])
print(Results[Mask])

In [ ]:
popt, pcov = curve_fit(lambda x,n: LinearFunction(x,-1.75,n),np.log(Results[Mask,1]),np.log(Results[Mask,2]),p0=[-1],maxfev=50000,bounds=([-np.inf],[np.inf]))
print(popt)
chi_0 = np.exp(popt[0])
print("chi_0={:.4f}".format(chi_0))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,3),dpi=200)
Ax[0,0].plot(Results[Mask,1],Results[Mask,2],'.')
Ax[0,0].plot(Results[Mask,1],PowerLaw(Results[Mask,1],chi_0,1.75))
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
Ax[0,0].set_xlabel(r'$  {\epsilon=\frac{|T-T_c|}{T_c}} $')
Ax[0,0].set_ylabel(r'${\chi}$')
Ax[0,0].text(0.5, 0.95,'$\gamma$={:.2f}, $\chi_0$={:.3e}'.format(1.75,chi_0), transform=Ax[0,0].transAxes, fontsize=11,verticalalignment='top')
Ax[0,0].set_title("Susceptibility scaling fit T>"+r"$T_{crit}$")
Fig.savefig("plots/susceptibility_scaling_fit_T>Tcrit.png",format='png',dpi=200)

In [ ]:
popt, pcov = curve_fit(lambda x,n: LinearFunction(x,-1.0,n),np.log(Results[Mask,1]),np.log(Results[Mask,3]),p0=[-3],maxfev=50000,bounds=([-np.inf],[np.inf]))
print(popt)
xi_0 = np.exp(popt[0])
print("xi_0={:.4f}".format(xi_0))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,3),dpi=200)
Ax[0,0].plot(Results[Mask,1],Results[Mask,3],'.')
Ax[0,0].plot(Results[Mask,1],PowerLaw(Results[Mask,1],xi_0,1))
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
Ax[0,0].set_xlabel(r'$\epsilon=\frac{|T-T_c|}{T_c}$')
Ax[0,0].set_ylabel(r'$\frac{\xi}{\sigma}$')
Ax[0,0].text(0.5, 0.95,'$\\nu$={:.2f}, $\\xi_0$={:.3e}'.format(1,xi_0), transform=Ax[0,0].transAxes, fontsize=11,verticalalignment='top')
Ax[0,0].set_title("Correlation length scaling fit T>"+r"$T_{crit}$")
#g.savefig("plots/corrlength_scaling_fit_T>Tcrit.png",format='png',dpi=200)

## Finite-size scaling of susceptibility

In [ ]:
def readFiniteSizeScalingSus(Density):
    Directory = getDirectoryPathFromDialog(dataPath)
    AllDirectories = glob.glob(Directory+"/N=*")
    Dict = {}
    for ParticleNumberDirectory in AllDirectories:
        N = int(re.search(r'(?<=N=)[\d]*',ParticleNumberDirectory).group())
        FileName = glob.glob(ParticleNumberDirectory+"/analyzed_data/structure_factor_from_fluctuations*")[0]
        StructureFactors = np.loadtxt(FileName, dtype=float, skiprows = 1)
        StructureFactors[:,1] = StructureFactors[:,1]/(StructureFactors[:,0]*Density)
        Dict[N] = StructureFactors
    SortedDict = {}
    for i in sorted(Dict):
        SortedDict[i] = Dict[i]
    return SortedDict

In [ ]:
Dict = readFiniteSizeScalingSus(0.75)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(7,4),dpi=100)
for N in Dict:
    Ax[0,0].plot(Dict[N][:,0], Dict[N][:,1],'.',label=N)

In [ ]:
TasLabel = {}
for T in Dict[500][:,0]:
    SusvsN = np.zeros((len(Dict),2))
    TasLabel[T] = SusvsN

TIndex = 0
for T in TasLabel:
    i = 0
    for N in Dict:
        TasLabel[T][i,0] = N
        TasLabel[T][i,1] = Dict[N][TIndex,1]
        i+=1
    TIndex+=1

In [ ]:
Fig, Ax = plt.subplots(len(TasLabel),1,squeeze=False, figsize=(8,5*len(TasLabel)),dpi=200)
i = 0
for T in TasLabel:
    Ax[i,0].plot(TasLabel[T][:,0], TasLabel[T][:,1],'.')
    Ax[i,0].set_title('T={}'.format(T))
    i+=1

In [ ]:
CorrelationLengths = Results[:,3][((np.array(Temperatures) > 0.77) & (np.array(Temperatures) < 0.85))]
print(CorrelationLengths)
BoxLengths = {  500:25.820,
               1000:36.515,
               2000:51.640,
               4000:73.030,
               8000:103.280,
              16000:146.059}
Tcrit = 0.7469
gamma = 1.75

In [ ]:
epsArray = np.zeros(len(TasLabel))
i = 0
for T in TasLabel:
    epsArray[i] = abs(T-Tcrit)/Tcrit
    i += 1

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(10,6),dpi=200)
i = 0
for N in Dict:
    Ax[0,0].plot((np.ones(len(Dict[N][:,0]))*BoxLengths[N])/CorrelationLengths, Dict[N][:,1]*(epsArray[:])**(gamma),'.',label=N)
    i+=1
#Ax[0,0].set_xlim(0,1)
#Ax[0,0].set_ylim(0,0.17)
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
Ax[0,0].legend()
Ax[0,0].set_title('Scaling plot of '+r'$\chi$')
Ax[0,0].set_xlabel(r'$y=\frac{\ell}{\xi}$')
Ax[0,0].set_ylabel(r'$\epsilon^{\gamma} \chi_{\ell}$')
#Fig.savefig('plots/Scaling_plot_of_susceptibility.png',format='png',dpi=200)

## Anomalous fit

In [ ]:
def anomalousOrnsteinZernike(k,rohchikBT,xi):
    return (rohchikBT*(1.0+(k*xi)**2)**(-7/8))

In [ ]:
def computeMultipleCorrelationLengthsAnomalous(Tc,kSquareMin,kSquareMax):
    Directory = getDirectoryPathFromDialog(dataPath)
    files = glob.glob(Directory+"/StructureFactors_*")
    Temperatures = []
    RedTemperatures = []
    Susceptibilities = []
    CorrelationLengths = []
    i = 0
    Fig, Ax = plt.subplots(len(files),2,squeeze=False, figsize=(14,70),dpi=100)
    Fig.subplots_adjust(hspace=0.3)
    for file in sorted(files):
        Temperature = float(re.search(r'(?<=T=)[\d]*.[\d]*',file).group())
        Temperatures.append(Temperature)
        RedTemperatures.append(abs(Temperature-Tc)/Tc)
        StructureFactorResults = readStructureFactorValues(file,1)
        y = (StructureFactorResults[:,4])**(-8.0/7.0)
        x = (StructureFactorResults[:,0])**2
        Mask = ((x > kSquareMin) & (x < kSquareMax))
        popt, pcov = curve_fit(LinearFunction,x[Mask],y[Mask],p0=[1,1],maxfev=50000,bounds=([0,0],[np.inf,np.inf]))
        Susceptibility = 1.0/(Temperature*(popt[1]**(7.0/8.0)))
        Susceptibilities.append(Susceptibility)
        CorrelationLength = np.sqrt(popt[0]/popt[1])
        CorrelationLengths.append(CorrelationLength)
        Ax[i,0].plot(x[Mask], y[Mask],'.')
        Ax[i,0].plot(x[Mask],LinearFunction(x[Mask],*popt),label=r'$\chi$='+str(Susceptibility)+r', $\frac{\xi}{\sigma}=$'+str(CorrelationLength))
        Ax[i,0].text(0.05, 0.75,'m={:.2f}, n={:.3e}'.format(popt[0],popt[1]), transform=Ax[i,0].transAxes, fontsize=14,verticalalignment='top')
        Ax[i,0].set_xlabel(r'$\left( k\sigma \right)^{2}$')
        Ax[i,0].set_ylabel(r'$\left( S_{cc}(k) \right)^{-8/7}$')
        Ax[i,0].set_title(r'T='+str(Temperature))
        Ax[i,0].legend(loc='upper left')
        Ax[i,1].plot(StructureFactorResults[:90,0],StructureFactorResults[:90,4],'.')
        Ax[i,1].plot(StructureFactorResults[:90,0],anomalousOrnsteinZernike(StructureFactorResults[:90,0],Temperature,Susceptibility,CorrelationLength))
        #Ax[i,0].set_xlim(0,1.5)
        #Ax[i,0].set_xscale('log')
        #Ax[i,0].set_yscale('log')
        i+=1
    return Temperatures, RedTemperatures, Susceptibilities, CorrelationLengths

In [ ]:
Temperatures, RedTemperatures, Susceptibilities, CorrelationLengths = computeMultipleCorrelationLengthsAnomalous(0.7469,0.04,0.15)

In [ ]:
Tcrit = 0.7469
Mask = np.array(Temperatures) > 0.8

In [ ]:
Results = np.column_stack([Temperatures,RedTemperatures,Susceptibilities,CorrelationLengths])
print(Results[Mask])

In [ ]:
popt, pcov = curve_fit(LinearFunction,np.log(Results[Mask,1]),np.log(Results[Mask,2]),p0=[-1.75,-1],maxfev=50000,bounds=([-2.751,-np.inf],[-1.75,np.inf]))
print(popt)
exponent=-popt[0]
chi_0 = np.exp(popt[1])
print("exponent={:.2f}, chi_0={:.4f}".format(exponent,chi_0))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,3),dpi=200)
Ax[0,0].plot(Results[Mask,1],Results[Mask,2],'.')
Ax[0,0].plot(Results[Mask,1],PowerLaw(Results[Mask,1],chi_0,exponent))
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
Ax[0,0].set_xlabel(r'$  {\epsilon=\frac{|T-T_c|}{T_c}} $')
Ax[0,0].set_ylabel(r'${\chi}$')

In [ ]:
popt, pcov = curve_fit(LinearFunction,np.log(Results[Mask,1]),np.log(Results[Mask,3]),p0=[-1.,-3],maxfev=50000,bounds=([-2.01,-np.inf],[-1.0,np.inf]))
print(popt)
exponent=-popt[0]
xi_0 = np.exp(popt[1])
print("exponent={:.2f}, xi_0={:.4f}".format(exponent,xi_0))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,3),dpi=200)
Ax[0,0].plot(Results[Mask,1],Results[Mask,3],'.')
Ax[0,0].plot(Results[Mask,1],PowerLaw(Results[Mask,1],xi_0,exponent))
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
Ax[0,0].set_xlabel(r'$\epsilon=\frac{|T-T_c|}{T_c}$')
Ax[0,0].set_ylabel(r'$\frac{\xi}{\sigma}$')

## Positions plotting

In [ ]:
APositions, BPositions = readParticleConfigFromDialog(9)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(7,7),dpi =200)
Host[0,0].plot(APositions[:,0],APositions[:,1],'.',color='b',ms=2)
Host[0,0].plot(BPositions[:,0],BPositions[:,1],'.',color='darkorange',ms=2)
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
Host[0,0].set_xlim(0,1)
Host[0,0].set_ylim(0,1)
#Host[0,0].set_title('config for T=0.84')
#Fig.savefig('Particle_config_T=0.84_structure_factors_not_equilibrated'+'.png',format='png')